# Installating R on WinPython

#### This procedure applys for Winpython  (Version of December 2015 and after) 
### 1 - Downloading and Installing R binary in the right place

In [ ]:
import os
import sys
import io

In [ ]:
os.environ["tmp_Rdirectory"]= "R" 
os.environ["R_HOME"] = os.environ["WINPYDIR"]+ "\\..\\tools\\R\\" 
os.environ["R_HOMEbin"]=os.environ["R_HOME"] + "bin" 
# for installation we need this
os.environ["tmp_Rbase"]=os.path.join(os.path.split(os.environ["WINPYDIR"])[0]  , 'tools','R' )  

In [ ]:
# downloading R-3.2.3 (65 380 480 Bytes, may takes a few minutes)
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2
if 'amd64' in sys.version.lower():
    r_comp ='/COMPONENTS="main,x64,translations'
else:
    r_comp ='/COMPONENTS="main,i386,translations'

# specify R binary and hash
r_url = "https://cran.r-project.org/bin/windows/base/R-3.3.1-win.exe"
hashes=("104bfc769d27c3fd3c9294cc4f82e232", "b1102b40c08914bada8f3240197ac960070c7db2" )

# specify target location
r_binary = os.path.basename(r_url)
r_installer = os.environ["WINPYDIR"]+"\\..\\tools\\"+r_binary
os.environ["r_installer"] = r_installer

# Download
g = urllib2.urlopen(r_url) 
with io.open(r_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
#checking it's there
!dir %r_installer%

In [ ]:
# checking it's the official R-3.2.3 
import hashlib
def give_hash(of_file, with_this):
    with io.open(r_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*12+"MD5"+" "*(32-12-3)+" "+" "*15+"SHA-1"+" "*(40-15-5)+"\n"+"-"*32+" "+"-"*40)

print ("%s %s %s" % (give_hash(r_installer, hashlib.md5) , give_hash(r_installer, hashlib.sha1),r_installer))
assert give_hash(r_installer, hashlib.md5) == hashes[0]
assert give_hash(r_installer, hashlib.sha1) == hashes[1]

In [ ]:
# let's install it 
# If you are "USB life style", or multi-winpython
#   ==> CLICK the OPTION "Don't create a StartMenuFolder' <== (when it will show up)
os.environ["tmp_R_comp"]=r_comp
!start cmd /C %r_installer% /DIR=%tmp_Rbase% %tmp_R_comp%

## During Installation (if you wan't to move the R installation after)

Choose non default option "Yes (customized startup"

then after 3 screens, Select  "Don't create a Start Menu Folder"

Un-select "Create a desktop icon"

Un-select "Save version number in registery"


<img src="https://raw.githubusercontent.com/stonebig/winpython_afterdoc/master/examples/images/r_setup_unclick_shortcut.GIF">

### 2 - create a R_launcher and install irkernel

In [ ]:
import os
import sys
import io
# let's create a R launcher  
r_launcher = r"""
@echo off
call %~dp0env.bat
rscript %*
"""
r_launcher_bat = os.environ["WINPYDIR"]+"\\..\\scripts\\R_launcher.bat"

# let's create a R init script
# in manual command line, you can use repos = c('http://irkernel.github.io/', getOption('repos'))
r_initialization = r"""
install.packages(c('repr', 'IRdisplay', 'stringr', 'crayon', 'pbdZMQ', 'devtools'), repos = c('http://cran.rstudio.com/', 'http://cran.rstudio.com/'))
devtools::install_github('IRkernel/IRkernel')
library('pbdZMQ')
library('repr')
library('IRkernel')
library('IRdisplay')
library('crayon')
library('stringr')
IRkernel::installspec()
"""
r_initialization_r = os.path.normpath(os.environ["WINPYDIR"]+"\\..\\scripts\\R_initialization.r")


for i in [(r_launcher,r_launcher_bat), (r_initialization, r_initialization_r)]:
    with io.open(i[1], 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in i[0].splitlines():
            f.write('%s\n' %  line  )


In [ ]:
#check what we are going to do 
print ("!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  " + r_initialization_r)

In [ ]:
# Launch Rkernel setup
os.environ["r_initialization_r"] = r_initialization_r
!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  %r_initialization_r%  

In [ ]:
# make RKernel a movable installation with the rest of WinPython 
from winpython import utils
base_winpython = os.path.dirname(os.path.normpath(os.environ["WINPYDIR"]))
rkernel_json=(base_winpython+"\\settings\\kernels\\ir\\kernel.json")

# so we get "argv": ["{prefix}/../tools/R/bin/x64/R"
utils.patch_sourcefile(rkernel_json, base_winpython.replace("\\","/"), r'{prefix}/..', silent_mode=False)

### 3- Install a R package via a IPython  Kernel

In [ ]:
%load_ext rpy2.ipython

#vitals: 'dplyr', 'R.utils', 'nycflights13'
# installation takes 2 minutes
%R install.packages(c('dplyr','R.utils', 'nycflights13'), repos='http://cran.rstudio.com/') 

### 4- Small demo via R magic

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library('dplyr')
library('nycflights13') 
write.csv(flights, "flights.csv")

In [ ]:
%R head(flights) 

In [ ]:
%R airports %>% mutate(dest = faa) %>% semi_join(flights) %>% head

### 5 - Installing the very best of R pakages (optional, you will start to get a really big directory)


In [ ]:
# essentials: 'tidyr', 'shiny', 'ggplot2', 'caret' , 'nnet'  
# remaining of Hadley Wickahm "stack" (https://github.com/rstudio)
%R install.packages(c('tidyr', 'ggplot2',  'shiny','caret' , 'nnet'), repos='https://cran.rstudio.com/') 
%R install.packages(c('knitr', 'purrr', 'readr', 'readxl'), repos='https://cran.rstudio.com/')
%R install.packages(c('rvest', 'lubridate', 'ggvis', 'readr','base64enc'), repos='https://cran.rstudio.com/')

# TRAINING = online training book http://r4ds.had.co.nz/ (or https://github.com/hadley/r4ds)

### 6 - Relaunch Jupyter Notebook to get a R kernel option
launch a new notebook of "R" type, and type in it:
    
library('dplyr')

library('nycflights13') 

head(flights)

### 9 - To Un-install  / Re-install R (or other trouble-shooting)

- launch  winpython**\tools\R\unins000.exe

- delete the directory winpython**\tools\R

- re-install
